# Peavey VIP-2 USB Scanner

![Peavey VIP-2](vip2-small.png)

Other relevant documentation includes:

* Peavey Vypyr Pro MIDI Specification (not entirely the same, but ideas are shared) https://assets.peavey.com/literature/additional/118745_31036.pdf
* My own footswitch controller for Arduino https://github.com/aughey/peavey_footswitch/blob/master/peavey_footswitch.ino
* Another DIY footswitch http://www.claytonfelt.com/peavey-vypyr-vip-amp-diy-midi-footswitch/
* Link to a reverse engineered MIDI spec for the commercial footswitch https://www.vguitarforums.com/smf/index.php?topic=3119.0

In [1]:
import mido
import sys
import time

Get Find the midi ports with VYPYR in in

In [2]:
input = None
output = None

for name in mido.get_output_names():
  if name.startswith("VYPYR"):
    output = mido.open_output(name)
    print("Output Opened: ",name)
    
for name in mido.get_input_names():
  if name.startswith("VYPYR"):
    input = mido.open_input(name)
    print("Input Opened:  ",name)


Output Opened:  VYPYR USB Interface:VYPYR USB Interface MIDI 1 20:0
Input Opened:   VYPYR USB Interface:VYPYR USB Interface MIDI 1 20:0


## Methods for manipulating the device over MIDI

In [3]:
prefix = [ 0x00, 0x00, 0x1B, 0x12, 0x00]
def send_control(id,value):
    output.send(msg = mido.Message("control_change", control=id, value=value))

def send_query():
    flush()
    msg = mido.Message('sysex', data= prefix + [0x63, 0x7f, 0x7f])
    output.send(msg)
    
def flush():
    for msg in input.iter_pending():
        None
        
def read_data():
    for msg in input.iter_pending():
       print(msg)
       print(msg.hex())
        
def read_block(type = None):
    for msg in input:
       if type == None or type == msg.type:
           return msg
        
def read_conf():
    send_query();
    return read_block(type='sysex')

def wait_for_data(t):
    start = time.time()
    while time.time() - start < t:
        msg = input.poll()
        print("a")
        if msg:
            return msg
    return None

read_data()
flush()

Mostly reference here for how to 

In [4]:
msg = mido.Message('note_on',note=0x10, channel=1)
output.send(msg);
msg = mido.Message('note_off',note=0x10, channel=1)
output.send(msg);


If we really need to write raw data, the _rt method is available, but thankfully this sysex approach is going to work.

In [5]:
output._rt

## Inquiry command - Get version

This sysex message is documented in the Vypyr Pro manual as an Inquiry command.  

The response to a standard inquiry is: F0 7E 7F 06 02 00 00 1B 30 00 00 00 ww xx yy zz F7

where:

'ww' is 41 ('A' amp mode), 44 ('D' demo mode), 54 ('T' tuner mode) or 42 ('B' bootcode -
software update mode)

'xx' is the major version number (30 or 31 for 0.0 or 1.0)

'yy' and 'zz' are the minor version digits (30..39 for a range of .00 to .99)

Perhaps this can be used to query the type of amp before sending other commands to check for compatibility.

In [6]:
#msg = mido.Message('sysex', data=[0x00,0x00,0x04,0x1b,0x12,0x00,0x04,0x63,0x7f,0x7f,0x05])
flush()
msg = mido.Message('sysex', data=[ 0x7E, 0x7F, 0x06, 0x01])
print(msg.hex())

F0 7E 7F 06 01 F7


In [7]:
output.send(msg)
inquiry_reply = read_block()
print(inquiry_reply.hex());

F0 7E 00 06 02 00 00 1B 12 00 09 00 30 31 36 31 F7


In [8]:
major = 0
wwxxyyzz = inquiry_reply.data[11:]
if wwxxyyzz[1] == 0x31:
    major = 1
minor = (wwxxyyzz[2] - 0x30) + (wwxxyyzz[3] - 0x30) * 10
print("Version: " + str(major) + "." + str(minor))

Version: 1.16


# Read the configuration block

In [9]:
send_query()
print(read_block())

sysex data=(0,0,27,18,9,99,127,127,1,10,0,2,0,2,0,2,0,2,0,2,0,2,0,6,0,12,0,2,0,2,3,7,3,7,0,2,2,8,0,2,0,0,0,10,0,2,0,2,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,15,0,0,0,0,0,0,0,0,4,12,4,5,4,1,4,4,2,0,4,7,4,1,4,9,4,14,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,2,0,10,5) time=0


## The result of this should be true.  Just reading the block twice

In [10]:
send_query()
a = read_block()
time.sleep(1)
send_query()
b = read_block()
a == b

True

We find a control by setting the value of the control to 1, reading it, changing the control value to 2, and reading
the configuration a second time.  One or more fields should change.  The changed field(s) are the values affected by this
control.

In [17]:
def find_control(id):
    send_control(id,1);
    a = read_conf()
    send_control(id,2);
    b = read_conf()
    i=0
    count = 0
    for v in zip(a.data,b.data):
        if v[0] != v[1]:
            print("  Index: ",i," ",hex(v[0])," != ",hex(v[1]))
            count += 1
        i=i+1
    return count

In [12]:
flush()
find_control(20)

  Index:  19   0x1  !=  0x2


True

In [13]:
find_control(21)

  Index:  35   0x1  !=  0x2


True

## Documentation:

Below are the control values for the knobs on the VIP-2.  These were determined by rotating knobs and watching the MIDI stream.

### Control Change

* pregain = 16
* low = 17
* mid = 18
* high = 19
* postgain = 20

* P1 = 27
* P2 = 26
* Delay Feedback = 21
* Delay Level = 23
* Reverb = 31

* Effect = 10
* Amp = 8
* Inst/Stomp = 11

![Front Panel](frontpanel.png)


In [14]:
known_controls = {
    "pregain" : 16,
    "low": 17,
    "mid" : 18,
    "high" : 19,
    "postgain" : 20,
    "P1" : 27,
    "P2" : 26,
    "Delay Feedback" : 21,
    "Delay Level" : 23,
    "Reverb" : 31,
    "Effect" : 10,
    "Amp" : 8,
    "Inst/Stomp" : 11
}

# Loop through all controls from 1 to 32 inclusive

Print out the configuration indices that change.  The controls that change should correspond to the known controls above.

More importantly, this searches for controls that are represented in the configuration that we do not have in our known
controls list.  Any control that hits this warning should be checked to see what it affects.

In [19]:
for control in range(1,32):
    print("Control: ",control)
    if find_control(control) > 0:
        # Make sure this is one of our known controls
        if not control in known_controls.values(): # Isn't that beatifully written?
            print("  Warning: this control changed something in the config, but isn't a known control.")

Control:  1
Control:  2
Control:  3
Control:  4
Control:  5
Control:  6
Control:  7
Control:  8
  Index:  8   0x2  !=  0x1
  Index:  9   0x3  !=  0xa
  Index:  10   0x6  !=  0x0
  Index:  11   0x4  !=  0x2
  Index:  12   0x3  !=  0x0
  Index:  13   0x5  !=  0x2
  Index:  14   0x6  !=  0x0
  Index:  15   0x4  !=  0x2
  Index:  16   0x5  !=  0x0
  Index:  17   0xa  !=  0x2
  Index:  18   0x6  !=  0x0
  Index:  19   0x4  !=  0x2
Control:  9
Control:  10
  Index:  25   0x1  !=  0xc
  Index:  26   0x3  !=  0x0
  Index:  27   0x4  !=  0x2
  Index:  28   0x5  !=  0x0
Control:  11
  Index:  8   0x0  !=  0x1
  Index:  9   0x8  !=  0xa
  Index:  10   0x6  !=  0x0
  Index:  11   0x4  !=  0x2
  Index:  12   0x3  !=  0x0
  Index:  13   0xf  !=  0x2
  Index:  14   0x5  !=  0x0
  Index:  15   0x5  !=  0x2
  Index:  17   0x0  !=  0x2
  Index:  18   0x7  !=  0x0
  Index:  19   0xf  !=  0x2
  Index:  43   0x4  !=  0xa
  Index:  44   0x5  !=  0x0
  Index:  45   0x4  !=  0x2
  Index:  46   0x2  !=  0x0
  

In [18]:
for control,control_id in known_controls.items():
    print("Control: ",control,"(",control_id,")")
    if 0 == find_control(control_id):
        print("  Warning: this is a known control, but we didn't see a change in the configuration")

Control:  Reverb ( 31 )
  Index:  21   0x1  !=  0x2
Control:  postgain ( 20 )
  Index:  19   0x1  !=  0x2
Control:  mid ( 18 )
  Index:  13   0x1  !=  0x2
Control:  P2 ( 26 )
  Index:  29   0x1  !=  0x2
Control:  Delay Feedback ( 21 )
  Index:  35   0x1  !=  0x2
Control:  P1 ( 27 )
  Index:  27   0x1  !=  0x2
Control:  Amp ( 8 )
  Index:  8   0x2  !=  0x1
  Index:  9   0x3  !=  0xa
  Index:  10   0x6  !=  0x0
  Index:  11   0x4  !=  0x2
  Index:  12   0x3  !=  0x0
  Index:  13   0x5  !=  0x2
  Index:  14   0x6  !=  0x0
  Index:  15   0x4  !=  0x2
  Index:  16   0x5  !=  0x0
  Index:  17   0xa  !=  0x2
  Index:  18   0x6  !=  0x0
  Index:  19   0x4  !=  0x2
Control:  pregain ( 16 )
  Index:  17   0x1  !=  0x2
Control:  high ( 19 )
  Index:  15   0x1  !=  0x2
Control:  Effect ( 10 )
  Index:  25   0x1  !=  0xc
  Index:  26   0x3  !=  0x0
  Index:  27   0x4  !=  0x2
  Index:  28   0x5  !=  0x0
Control:  Delay Level ( 23 )
  Index:  39   0x1  !=  0x2
Control:  Inst/Stomp ( 11 )
  Index:  8